<a href="https://colab.research.google.com/github/luanakwon/ML_Pytorch_pre2022/blob/main/BabyNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Seems weird, but here is the idea  
It is a supervised learning, except question and answer is provided sequentially, like you would when teach babies how to speak.

but didn't go very well.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optimizer


import numpy as np
import random

In [ ]:
a = torch.Tensor((1,2,3,4))
b = torch.Tensor((5,5,5,5))
print(torch.cat((a,b)))

tensor([1., 2., 3., 4., 5., 5., 5., 5.])


In [ ]:
im_list = ['참외', '딸기', '사과', '키위', '겨울배', '타지리', '게', '노인', '광고', '개입']
wd_list = ['','이건','아니야','맞아','뭐야','참외', '딸기', '사과', '키위', '겨울배', '타지리', '게', '노인', '광고', '개입']

epoch = 100

image_len = len(im_list)
word_len = len(wd_list)
v1_len = 32
v2_len = 32
S_ratio = 0.1
L_ratio = 0.7

In [ ]:
class Image2Vec(nn.Module):
  def __init__(self):
    super(Image2Vec,self).__init__()
    
    self.layer = nn.Sequential(
        nn.Linear(image_len,v1_len),
        nn.ReLU()
    )

  def forward(self, x):
    return self.layer(x)

In [ ]:
class Word2Vec(nn.Module):
  def __init__(self):
    super(Word2Vec,self).__init__()

    self.layer = nn.Sequential(
        nn.Linear(word_len,v2_len),
        nn.ReLU()
    )

  def forward(self,x):
    return self.layer(x)

In [ ]:
class Vec2Word(nn.Module):
  def __init__(self):
    super(Vec2Word,self).__init__()

    self.layer = nn.Sequential(
        nn.Linear(2*(v1_len+v2_len),word_len),
        nn.Sigmoid()
    )
  
  def forward(self,x):
    x = x.view(-1)
    return self.layer(x)

In [ ]:
class ThinkNN(nn.Module):
  def __init__(self):
    super(ThinkNN, self).__init__()

    self.short_term = nn.Sequential(
        nn.Linear(v1_len+v2_len,v1_len+v2_len),
        nn.ReLU()
    )

    self.long_term = nn.Sequential(
        nn.Linear(v1_len+v2_len,v1_len+v2_len),
        nn.ReLU()
    )

  def forward(self, x, mem):
    mem = mem.view(2,-1)
    short = torch.matmul(torch.Tensor((1,0)).view(1,2),mem).view(-1)
    long = torch.matmul(torch.Tensor((0,1)).view(1,2),mem).view(-1)

    short = self.short_term(short*S_ratio+x*(1-S_ratio))
    long = self.long_term(long*L_ratio + x*(1-L_ratio))

    return torch.stack((short,long))

In [ ]:
def init_weights(m):
  if type(m) == nn.Linear:
    init.normal_(m.weight.data,std=0.02)

In [ ]:
nn1 = Image2Vec()
nn2 = Word2Vec()
nn3 = ThinkNN()
nn4 = Vec2Word()

nn1.apply(init_weights)
nn2.apply(init_weights)
nn3.apply(init_weights)
nn4.apply(init_weights)

# o1 = optimizer.SGD(nn1.parameters(),lr=0.001)
# o2 = optimizer.SGD(nn2.parameters(),lr=0.001)
o3 = optimizer.SGD(nn3.parameters(),lr=0.001)
o4 = optimizer.SGD(nn4.parameters(),lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [ ]:
def Baby(image, word, mem):
  im_onehot = np.zeros(image_len)
  im_onehot[image] = 1
  im_onehot = torch.Tensor(im_onehot)
  
  wd_onehot = np.zeros(word_len)
  wd_onehot[word] = 1
  wd_onehot = torch.Tensor(wd_onehot)

  v1 = nn1.forward(im_onehot)
  v2 = nn2.forward(wd_onehot)
  v3 = torch.cat((v1,v2))

  mem = nn3.forward(v3,mem)

  word_out = nn4.forward(mem)

  return (mem, word_out)

In [ ]:
print(torch.Tensor([1]))

tensor([1.])


In [ ]:
mem = init.normal_(torch.zeros((v1_len+v2_len)*2))

for e in range(10):
  o3.zero_grad()
  o4.zero_grad()

  image = random.randint(0,image_len-1)
  word = 1
  print(f'image = {im_list[image]}\n',wd_list[word],end=' ')
  mem, word_out = Baby(image,word,mem)

  word = image + 5
  print(wd_list[word])
  mem, word_out = Baby(image,word,mem)
  ans = np.argmax(word_out.detach().numpy())
  print(wd_list[ans])

  loss = loss_func(word_out.view(1,-1),torch.LongTensor([word]))
  loss.backward()
  o3.step()
  o4.step()
  mem = mem.detach()

  o3.zero_grad()
  o4.zero_grad()

  for i in range(30):
    if word == ans:
      word = 3
      print(wd_list[word])
      break
    else:
      word = 2
      print(wd_list[word],end=' ')
      mem, word_out = Baby(image,word,mem)
      word = image+5
      print(wd_list[word])
      mem, word_out = Baby(image,word,mem)
      ans = np.argmax(word_out.detach().numpy())
      print(wd_list[ans], loss.data)
      loss = loss_func(word_out.view(1,-1),torch.LongTensor([word]))
      loss.backward()
      o3.step()
      o4.step()
      mem = mem.detach()
      



image = 노인
 이건 노인
아니야
아니야 노인
아니야 tensor(2.7080)
아니야 노인
아니야 tensor(2.7075)
아니야 노인
아니야 tensor(2.7074)
아니야 노인
아니야 tensor(2.7072)
아니야 노인
아니야 tensor(2.7070)
아니야 노인
아니야 tensor(2.7066)
아니야 노인
아니야 tensor(2.7062)
아니야 노인
아니야 tensor(2.7057)
아니야 노인
아니야 tensor(2.7052)
아니야 노인
아니야 tensor(2.7045)
아니야 노인
아니야 tensor(2.7037)
아니야 노인
아니야 tensor(2.7029)
아니야 노인
아니야 tensor(2.7020)
아니야 노인
아니야 tensor(2.7010)
아니야 노인
아니야 tensor(2.6999)
아니야 노인
아니야 tensor(2.6988)
아니야 노인
아니야 tensor(2.6975)
아니야 노인
아니야 tensor(2.6962)
아니야 노인
아니야 tensor(2.6948)
아니야 노인
아니야 tensor(2.6933)
아니야 노인
아니야 tensor(2.6917)
아니야 노인
아니야 tensor(2.6901)
아니야 노인
아니야 tensor(2.6883)
아니야 노인
노인 tensor(2.6865)
맞아
image = 노인
 이건 노인
노인
맞아
image = 사과
 이건 사과
노인
아니야 사과
노인 tensor(2.7248)
아니야 사과
노인 tensor(2.7248)
아니야 사과
노인 tensor(2.7247)
아니야 사과
노인 tensor(2.7245)
아니야 사과
노인 tensor(2.7243)
아니야 사과
노인 tensor(2.7239)
아니야 사과
노인 tensor(2.7235)
아니야 사과
노인 tensor(2.7230)
아니야 사과
노인 tensor(2.7224)
아니야 사과
노인 tensor(2.7218)
아니야 사과
노인 tensor(2.7210)
아니야 사과
노인 tensor(2.7202)
아니야 사과
